In [7]:
import pandas as pd
import numpy as np
import os

# 개요

생활인구수를 변수로 추가하고자 한다.

## 계획

1. 사용할 데이터와 변수 탐색 <br>
1) 경매 데이터<br>
: 주소변수를 사용하여 법정동명변수(시, 구, 동) 생성<br>
2) 법정동코드 데이터<br>
: 법정동명, 법정동코드<br>
3) 법정동행정동 맵핑된 데이터<br>
: 행정동코드, 법정동코드<br>
4) 생활인구데이터<br>
: 행정동코드, 생활인구수<br>


2. merge 단계<br>
1) merge_1<br>
: 법정동코드 데이터와 행정동코드 데이터 merge<br>
: key = 법정동코드<br>
2) merge_2<br>
: merge_1 데이터와 생활인구데이터 merge<br>
: key = 행정동코드<br>
3) merge_3<br>
: merge_2데이터와 경매 데이터<br>
: key = 법정동명<br>

# 데이터 읽어오기

In [8]:
# 경매 크롤링
auction = pd.read_csv('1_상가경매물건.csv', header='infer', engine='python', encoding='utf-8-sig', sep=',')

# 법정동 코드
dong_code = pd.read_csv('2_법정동코드.csv', engine='python', header='infer', encoding = 'euc-kr')

# 법정동 행정동 코드 맵핑
dong_mapping = pd.read_csv('3_code.csv', engine='python', header='infer', encoding='euc-kr')

# 생활인구
population = pd.read_csv('4_생활인구.csv', engine='python', header='infer', encoding='UTF-8')

In [9]:
auction.head(1)

,경매구분,매각기일,주소,도로명주소,우편번호,용도,대지권,전용면적,매각대상,사건접수날짜,감정가,최저가,청구금액,개시결정,기일현황,물건현황_토지이용계획,감정평가현황,임차인_대항력여부,등기사항_소멸여부,상세페이지링크
0,임의,2018-08-14,서울특별시 은평구 대조동 240 팜스퀘어 제7층 제씨-055호,(03397) 서울특별시 은평구 대조동 240 팜스퀘어 제7층 제씨-055호\n[도...,3397.0,상가(점포),1.13㎡ (0.34평),4.97㎡ (1.5평),토지/건물일괄매각,2016-11-29,"72,000,000원","(17%) 12,079,000원","19,500,000원",2016-11-30,"회차 매각기일 최저매각금액 결과\n신건 2017-10-17 72,000,000원 유...","지하철3,6호선 불광역 남서측 인근 불광로 대로변에 위치\n주위는 통일로,불광로,진...","감정평가현황 강남감정\n가격시점 2016-12-20\n감정가 72,000,000원 ...",배당종기일: 2017-02-10\n이OOOO 없음\n사업 : 없음\n확정 : 없음\...,소유권 이전\n 2005-08-17 집합\n 태OOOO\n보존\n소유권 이전\n...,http://www.speedauction.co.kr/v3/M_view/info_r...


In [10]:
dong_code.head(1)

,법정동코드,법정동명
0,1111010100,서울특별시 종로구 청운동


In [11]:
dong_mapping.head(1)

,행정동코드,법정동코드
0,1111051500,1111010100


In [12]:
population.head(1)

,기준일ID,시간대구분,행정동코드,총생활인구수
0,20180701,0,11215760,19006.4504


# 데이터 맵핑

## auction데이터와 맵핑할 데이터 만들기

### 법정동코드 + 행정동코드

In [13]:
merge_1 = pd.merge(dong_code, dong_mapping, how = 'left', on = '법정동코드')
merge_1

,법정동코드,법정동명,행정동코드
0,1111010100,서울특별시 종로구 청운동,1111051500
1,1111010200,서울특별시 종로구 신교동,1111051500
2,1111010300,서울특별시 종로구 궁정동,1111051500
3,1111010400,서울특별시 종로구 효자동,1111051500
4,1111010500,서울특별시 종로구 창성동,1111051500
5,1111010600,서울특별시 종로구 통의동,1111053000
6,1111010700,서울특별시 종로구 적선동,1111053000
7,1111010800,서울특별시 종로구 통인동,1111051500
8,1111010900,서울특별시 종로구 누상동,1111051500
9,1111011000,서울특별시 종로구 누하동,1111051500


In [14]:
merge_1['행정동코드'] = merge_1['행정동코드']//100
merge_1

,법정동코드,법정동명,행정동코드
0,1111010100,서울특별시 종로구 청운동,11110515
1,1111010200,서울특별시 종로구 신교동,11110515
2,1111010300,서울특별시 종로구 궁정동,11110515
3,1111010400,서울특별시 종로구 효자동,11110515
4,1111010500,서울특별시 종로구 창성동,11110515
5,1111010600,서울특별시 종로구 통의동,11110530
6,1111010700,서울특별시 종로구 적선동,11110530
7,1111010800,서울특별시 종로구 통인동,11110515
8,1111010900,서울특별시 종로구 누상동,11110515
9,1111011000,서울특별시 종로구 누하동,11110515


### 법정동코드 + 행정동코드 + 생활인구

#### 생활인구 데이터 reshape

In [15]:
population.tail()

,기준일ID,시간대구분,행정동코드,총생활인구수
1709563,20181231,23,11740650,33671.9255
1709564,20181231,23,11740660,31463.2373
1709565,20181231,23,11740685,65983.8447
1709566,20181231,23,11740690,7672.5173
1709567,20181231,23,11740700,26374.5982


In [16]:
population['행정동코드'].unique

<bound method Series.unique of 0          11215760
1          11710632
2          11590630
3          11590660
4          11590650
5          11410555
6          11350600
7          11590651
8          11305610
9          11680700
10         11380580
11         11320670
12         11590670
13         11620545
14         11215740
15         11545610
16         11350670
17         11620625
18         11620605
19         11290770
20         11650520
21         11710680
22         11290630
23         11680655
24         11620665
25         11440555
26         11350611
27         11215710
28         11650581
29         11170590
             ...   
1709538    11710631
1709539    11710632
1709540    11710641
1709541    11710642
1709542    11710646
1709543    11710647
1709544    11710650
1709545    11710670
1709546    11710680
1709547    11710690
1709548    11710710
1709549    11710720
1709550    11740515
1709551    11740520
1709552    11740530
1709553    11740540
1709554    11740550
1709555  

In [17]:
population_1 = population.drop(['기준일ID', '시간대구분'], axis=1)

In [18]:
population_gruop = population_1.groupby("행정동코드").mean()
population_gruop

,총생활인구수
행정동코드,
11110515,16551.054077
11110530,31276.468667
11110540,6941.842364
11110550,12913.703762
11110560,19744.041674
11110570,7243.537884
11110580,8771.958811
11110600,8088.628950
11110615,71579.869659


In [19]:
print(type(population_gruop))

<class 'pandas.core.frame.DataFrame'>


#### reshape한 생활인구 데이터로 merge

In [20]:
merge_2 = pd.merge(merge_1, population_gruop, how='left', on = '행정동코드')
merge_2

,법정동코드,법정동명,행정동코드,총생활인구수
0,1111010100,서울특별시 종로구 청운동,11110515,16551.054077
1,1111010200,서울특별시 종로구 신교동,11110515,16551.054077
2,1111010300,서울특별시 종로구 궁정동,11110515,16551.054077
3,1111010400,서울특별시 종로구 효자동,11110515,16551.054077
4,1111010500,서울특별시 종로구 창성동,11110515,16551.054077
5,1111010600,서울특별시 종로구 통의동,11110530,31276.468667
6,1111010700,서울특별시 종로구 적선동,11110530,31276.468667
7,1111010800,서울특별시 종로구 통인동,11110515,16551.054077
8,1111010900,서울특별시 종로구 누상동,11110515,16551.054077
9,1111011000,서울특별시 종로구 누하동,11110515,16551.054077


## 경매데이터 주소_분할 -> 법정동주소 생성

In [21]:
adress = auction.주소
for idx, i in enumerate(adress) :
    str = i
    str_split = str.split(' ')
    si = str_split[0]
    gu = str_split[1]
    dong = str_split[2]
    adress[idx] = si + " " + gu + " " + dong
auction["법정동명"] = adress

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
auction.head(1)

,경매구분,매각기일,주소,도로명주소,우편번호,용도,대지권,전용면적,매각대상,사건접수날짜,...,최저가,청구금액,개시결정,기일현황,물건현황_토지이용계획,감정평가현황,임차인_대항력여부,등기사항_소멸여부,상세페이지링크,법정동명
0,임의,2018-08-14,서울특별시 은평구 대조동,(03397) 서울특별시 은평구 대조동 240 팜스퀘어 제7층 제씨-055호\n[도...,3397.0,상가(점포),1.13㎡ (0.34평),4.97㎡ (1.5평),토지/건물일괄매각,2016-11-29,...,"(17%) 12,079,000원","19,500,000원",2016-11-30,"회차 매각기일 최저매각금액 결과\n신건 2017-10-17 72,000,000원 유...","지하철3,6호선 불광역 남서측 인근 불광로 대로변에 위치\n주위는 통일로,불광로,진...","감정평가현황 강남감정\n가격시점 2016-12-20\n감정가 72,000,000원 ...",배당종기일: 2017-02-10\n이OOOO 없음\n사업 : 없음\n확정 : 없음\...,소유권 이전\n 2005-08-17 집합\n 태OOOO\n보존\n소유권 이전\n...,http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 은평구 대조동


## 법정동 + 행정동 + 생활인구 + 경매

In [23]:
merge_3 = pd.merge(auction, merge_2, how = 'left', on = '법정동명')
merge_3

,경매구분,매각기일,주소,도로명주소,우편번호,용도,대지권,전용면적,매각대상,사건접수날짜,...,기일현황,물건현황_토지이용계획,감정평가현황,임차인_대항력여부,등기사항_소멸여부,상세페이지링크,법정동명,법정동코드,행정동코드,총생활인구수
0,임의,2018-08-14,서울특별시 은평구 대조동,(03397) 서울특별시 은평구 대조동 240 팜스퀘어 제7층 제씨-055호\n[도...,3397.0,상가(점포),1.13㎡ (0.34평),4.97㎡ (1.5평),토지/건물일괄매각,2016-11-29,...,"회차 매각기일 최저매각금액 결과\n신건 2017-10-17 72,000,000원 유...","지하철3,6호선 불광역 남서측 인근 불광로 대로변에 위치\n주위는 통일로,불광로,진...","감정평가현황 강남감정\n가격시점 2016-12-20\n감정가 72,000,000원 ...",배당종기일: 2017-02-10\n이OOOO 없음\n사업 : 없음\n확정 : 없음\...,소유권 이전\n 2005-08-17 집합\n 태OOOO\n보존\n소유권 이전\n...,http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 은평구 대조동,1138010600,11380570,34524.416664
1,임의,2018-08-14,서울특별시 구로구 구로동,(08297) 서울특별시 구로구 구로동 49 구로동금호아파트 제2층 제204호\n[...,8297.0,오피스텔(업무),20.19㎡ (6.11평),106.59㎡ (32.24평),토지/건물일괄매각,2018-02-05,...,"회차 매각기일 최저매각금액 결과\n신건 2018-08-14 271,000,000원 ...","구로고등학교 북동측 인근에 위치\n주위는 주상복합건물 및 각종 근린생활시설, 아파트...","감정평가현황 국보감정\n가격시점 2018-02-20\n감정가 271,000,000원...",배당종기일: 2018-05-09\n법OOOOO 없음\n사업 : 2013-07-12\...,"(근)저당 소멸기준\n 2010-02-10 집합\n 중OOOOO\n 96,00...",http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 구로구 구로동,1153010200,11530520,16438.966397
2,임의,2018-08-14,서울특별시 구로구 구로동,(08297) 서울특별시 구로구 구로동 49 구로동금호아파트 제2층 제204호\n[...,8297.0,오피스텔(업무),20.19㎡ (6.11평),106.59㎡ (32.24평),토지/건물일괄매각,2018-02-05,...,"회차 매각기일 최저매각금액 결과\n신건 2018-08-14 271,000,000원 ...","구로고등학교 북동측 인근에 위치\n주위는 주상복합건물 및 각종 근린생활시설, 아파트...","감정평가현황 국보감정\n가격시점 2018-02-20\n감정가 271,000,000원...",배당종기일: 2018-05-09\n법OOOOO 없음\n사업 : 2013-07-12\...,"(근)저당 소멸기준\n 2010-02-10 집합\n 중OOOOO\n 96,00...",http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 구로구 구로동,1153010200,11530530,37379.210128
3,임의,2018-08-14,서울특별시 구로구 구로동,(08297) 서울특별시 구로구 구로동 49 구로동금호아파트 제2층 제204호\n[...,8297.0,오피스텔(업무),20.19㎡ (6.11평),106.59㎡ (32.24평),토지/건물일괄매각,2018-02-05,...,"회차 매각기일 최저매각금액 결과\n신건 2018-08-14 271,000,000원 ...","구로고등학교 북동측 인근에 위치\n주위는 주상복합건물 및 각종 근린생활시설, 아파트...","감정평가현황 국보감정\n가격시점 2018-02-20\n감정가 271,000,000원...",배당종기일: 2018-05-09\n법OOOOO 없음\n사업 : 2013-07-12\...,"(근)저당 소멸기준\n 2010-02-10 집합\n 중OOOOO\n 96,00...",http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 구로구 구로동,1153010200,11530540,45771.611272
4,임의,2018-08-14,서울특별시 구로구 구로동,(08297) 서울특별시 구로구 구로동 49 구로동금호아파트 제2층 제204호\n[...,8297.0,오피스텔(업무),20.19㎡ (6.11평),106.59㎡ (32.24평),토지/건물일괄매각,2018-02-05,...,"회차 매각기일 최저매각금액 결과\n신건 2018-08-14 271,000,000원 ...","구로고등학교 북동측 인근에 위치\n주위는 주상복합건물 및 각종 근린생활시설, 아파트...","감정평가현황 국보감정\n가격시점 2018-02-20\n감정가 271,000,000원...",배당종기일: 2018-05-09\n법OOOOO 없음\n사업 : 2013-07-12\...,"(근)저당 소멸기준\n 2010-02-10 집합\n 중OOOOO\n 96,00...",http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 구로구 구로동,1153010200,11530550,18470.907088
5,임의,2018-08-14,서울특별시 구로구 구로동,(08297) 서울특별시 구로구 구로동 49 구로동금호아파트 제2층 제204호\n[...,8297.0,오피스텔(업무),20.19㎡ (6.11평),106.59㎡ (32.24평),토지/건물일괄매각,2018-02-05,...,"회차 매각기일 최저매각금액 결과\n신건 2018-08-14 271,000,000원 ...","구로고등학교 북동측 인근에 위치\n주위는 주상복합건물 및 각종 근린생활시설, 아파트...","감정평가현황 국보감정\n가격시점 2018-02-20\n감정가 271,000,000원...",배당종기일: 2018-05-09\n법OOOOO 없음\n사업 : 2013-07-12\...,"(근)저당 소멸기준\n 2010-02-10 집합\n 중OOOOO\n 96,00...",http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 구로구 구로동,1153010200,11530560,37840.290837
6,임의,2018-08-27,서울특별시 노원구 중계동,(01734) 서울특별시 노원구 중계동 364-17 유경데파트 제지하1층 제비10호...,1734.0,상가(점포),3.25㎡ (0.98평),10.38㎡ (3.14평),토지/건물일괄매각,2018-01-26,...,"회차 매각기일 최저매각금액 결과\n신건 2018-06-18 22,000,000원 유...","불암고등학교 서측 인근에 위치\n주위는 근린생활시설, 대규모 아파트단지 등이 소재\...","감정평가현황 해랑감정\n가격시점 2018-02-23\n감정가 22,000,000원 ...",배당종기일: 2018-04-19\n조OO 없음\n사업 : 없음\n확정 : 없음\n배...,소유권 이전\n 2000-06-21 집합\n 이OO\n매매\n(근)저당 소멸기준...,http://www.speedauction.co.kr/v3/M_view/info_r...,서울특별시 노원구 중계동,1135010600,11350619,21151.426980
7,임의,2018-08-27,서울특별시 노원구 중계동,(01734) 서울특별시 노원구 중계동 364-17 유경데파트 제지하1층 제비10호...,1734.0,상가(점포),3.25㎡ (0.98평),10.38㎡ (3.14평),토지/건물일괄매각,2018-01-26,...,"회차 매각기일 최저매각금액 결과\n신건 2018-06-18 22,000,000원 유...","불암고등학교 서측 인근에 위치\n주위는 근린생활시설, 대규모 아파트단지 등이 소재\...","감정평가현황 해랑감정\n가격시점 2018-02-23\n감정가 22,000,000원 ...",배당종기일: 2018-04-19\n조OO 없음\n사업 : 없음\n확정 : 없음\n배

# 결론

=>행정동코드가 법정동코드보다 단위가 커서 생활인구변수를 디테일한 값으로 구할 수 없었다.

더미로 사용하거나, 다른 아이디어가 필요(예: 상권데이터의 상세주소로 유동인구 데이터를 가져와보기)

## 이슈
경매데이터는 226개의 행이었으나 merge의 결과로 664개의 행이 됨. 즉 중복되는 행이 생김. (행정동코드와 법정동 코드의 기준(?)이 달라서 생긴 결과로 봄, 글로 깔끔하게 설명하기가 어려움.)

어떻게 해결하면 좋을지 아이디어 공유가 필요

csv파일로 저장

In [25]:
merge_3.to_csv('경매_생활인구_맵핑.csv', header='infer', encoding='utf-8-sig', sep=',')